In [1]:
%matplotlib inline
import pandas as pd
import os
# Import the main functionality from the SimFin Python API.
import simfin as sf

# Import names used for easy access to SimFin's data-columns.
from simfin.names import *

import yahoo_fin.stock_info as si
# import pandas_datareader.data as web

from dateutil.relativedelta import relativedelta
from datetime import datetime

In [ ]:
# sample base
tickers = si.tickers_sp500()

In [ ]:
cwd = os.getcwd()
cwd

In [ ]:
sf.set_data_dir('/Users/feiyiyang/Documents/Bootcamp/Team9-Project2/Bootcamp-Project2/simfin_data/')

In [ ]:
sf.load_api_key(path='/Users/feiyiyang/Documents/Bootcamp/Team9-Project2/Bootcamp-Project2/simfin_data/simfin_api_key.txt')

### Part 1: Grab quarterly fundamentals from Simfin API

In [ ]:
df_income = sf.load(dataset='income', variant='quarterly', market='us')
df_balance = sf.load(dataset='balance', variant='quarterly', market='us')
df_cashflow = sf.load(dataset='cashflow', variant='quarterly', market='us')

In [ ]:
df_cashflow.columns

In [ ]:
df_fundamentals = pd.merge(df_income[['Ticker','Fiscal Year','Fiscal Period','Report Date', 'Publish Date','Revenue','Gross Profit','Operating Income (Loss)','Net Income']], 
                           df_balance[['Ticker','Fiscal Year','Fiscal Period','Shares (Basic)','Cash, Cash Equivalents & Short Term Investments','Total Assets','Total Liabilities','Total Equity']],  
                           how='left', 
                           on=['Ticker','Fiscal Year','Fiscal Period'])

In [ ]:
df_fundamentals= pd.merge(df_fundamentals,
                          df_cashflow[['Ticker','Fiscal Year','Fiscal Period','Net Cash from Operating Activities','Net Change in Long Term Investment','Net Cash from Investing Activities', 'Dividends Paid', 'Net Cash from Financing Activities', 'Net Change in Cash']],
                           how='left', 
                           on=['Ticker','Fiscal Year','Fiscal Period'])

In [ ]:
df_fundamentals.tail()

In [ ]:
# add a column for next earning date - to set the start/end date for append daily price
df_fundamentals['Next_publish_date']=df_fundamentals.groupby("Ticker")['Publish Date'].shift(-1)
u = (pd.to_datetime(df_fundamentals['Publish Date']) + pd.DateOffset(months=3)).dt.date
df_fundamentals.Next_publish_date.fillna(u,inplace=True)

In [ ]:
df_fundamentals.to_csv('data/us_fundamental.csv',index=False)

### Part 2: Grab EARNING DATA from yahoo finance API (yahoo_fin.stock_info)

In [126]:
# load fundamental data
df_fundamentals=pd.read_csv('data/us_fundamental.csv')

In [132]:
dftest=pd.DataFrame.from_dict(si.get_earnings_history("A"))[['ticker','startdatetime','epsestimate','epsactual','epssurprisepct']]
dftest.startdatetime=pd.to_datetime(dftest.startdatetime).dt.date
dftest=dftest[dftest['startdatetime']>pd.to_datetime("2016-1-1").date()]
dftest['earning_y']= pd.DatetimeIndex(dftest['startdatetime']).year
dftest['earning_m']= pd.DatetimeIndex(dftest['startdatetime']).month
dftest.sort_values(by=['startdatetime'],inplace=True)
dftest

,ticker,startdatetime,epsestimate,epsactual,epssurprisepct,earning_y,earning_m
26,A,2016-02-16,0.43,0.46,6.24,2016,2
25,A,2016-05-16,0.39,0.44,13.11,2016,5
24,A,2016-08-17,0.47,0.49,4.70,2016,8
23,A,2016-11-15,0.52,0.59,13.24,2016,11
22,A,2017-02-14,0.49,0.53,8.16,2017,2
21,A,2017-05-22,0.48,0.58,19.83,2017,5
20,A,2017-08-15,0.52,0.59,13.03,2017,8
19,A,2017-11-20,0.62,0.67,7.37,2017,11
18,A,2018-02-14,0.58,0.66,13.79,2018,2
17,A,2018-05-14,0.64,0.65,1.25,2018,5


In [134]:
df1=df_fundamentals[df_fundamentals['Ticker']=='A'][['Ticker','Publish Date']]
df1['Publish_y']=pd.DatetimeIndex(df1['Publish Date']).year
df1['Publish_m']=pd.DatetimeIndex(df1['Publish Date']).month
df1

,Ticker,Publish Date,Publish_y,Publish_m
0,A,2016-09-07,2016,9
1,A,2016-12-20,2016,12
2,A,2017-03-08,2017,3
3,A,2017-06-06,2017,6
4,A,2017-09-06,2017,9
5,A,2017-12-21,2017,12
6,A,2018-03-06,2018,3
7,A,2018-05-31,2018,5
8,A,2018-08-30,2018,8
9,A,2018-12-20,2018,12


In [136]:
merge=pd.merge(df1,dftest,how='left',left_on=('Ticker','Publish_y'),right_on=('ticker','earning_y'))
merge

,Ticker,Publish Date,Publish_y,Publish_m,ticker,startdatetime,epsestimate,epsactual,epssurprisepct,earning_y,earning_m
0,A,2016-09-07,2016,9,A,2016-02-16,0.43,0.46,6.24,2016,2
1,A,2016-09-07,2016,9,A,2016-05-16,0.39,0.44,13.11,2016,5
2,A,2016-09-07,2016,9,A,2016-08-17,0.47,0.49,4.70,2016,8
3,A,2016-09-07,2016,9,A,2016-11-15,0.52,0.59,13.24,2016,11
4,A,2016-12-20,2016,12,A,2016-02-16,0.43,0.46,6.24,2016,2
...,...,...,...,...,...,...,...,...,...,...,...
75,A,2021-03-02,2021,3,A,2021-11-22,1.18,1.21,2.98,2021,11
76,A,2021-06-01,2021,6,A,2021-02-16,0.89,1.06,18.57,2021,2
77,A,2021-06-01,2021,6,A,2021-05-25,0.83,0.97,17.29,2021,5
78,A,2021-06-01,2021,6,A,2021-08-17,0.99,1.10,10.89,2021,8


In [ ]:
merge= merge[(merge.earning_m == merge.Publish_m) | (merge.earning_m == merge.Publish_m-1)]

In [ ]:
merge

In [ ]:
earningdata=[]
for ticker in tickers_sp500:
    dftest=pd.DataFrame.from_dict(si.get_earnings_history(ticker))[['ticker','startdatetime','epsestimate','epsactual','epssurprisepct']]
    dftest.startdatetime=pd.to_datetime(dftest.startdatetime).dt.date
    dftest=dftest[dftest['startdatetime']>pd.to_datetime("2016-1-1").date()]
    dftest['earningestimate_year']= pd.DatetimeIndex(dftest['startdatetime']).year
    dftest['earningestimate_month']= pd.DatetimeIndex(dftest['startdatetime']).month
    dftest.sort_values(by=['startdatetime'],inplace=True)
    
    df1=df_fundamentals[df_fundamentals['Ticker']==ticker][['Ticker']]
    

### Part 3: Grab daily stock price from yahoo finance API (yahoo_fin.stock_info)

In [ ]:
# load fundamental data
df_fundamentals=pd.read_csv('data/us_fundamental.csv')

In [ ]:
# sample base and starting date for training
tickers_sp500 = list(si.tickers_sp500())
start_date = '2016-01-01'
end_date = '2021-12-31'

In [ ]:
# append daily price data to fundamental where the date is within the current publish date and next publish date
frames=[]
for ticker in tickers_sp500:
    try:
        df1=df_fundamentals[df_fundamentals['Ticker']==ticker]
        df2=si.get_data(ticker , start_date = start_date, end_date=end_date,index_as_date=False)
    except:
        print(f"No data available for {ticker}")
    else:
        df_merge=pd.merge(df1,df2,how='left',left_on='Ticker',right_on='ticker')
        df_merge= df_merge[(df_merge.date > df_merge['Publish Date']) & (df_merge.date <= df_merge['Next_publish_date'])]
        frames.append(df_merge)
    

df_sp500 = pd.concat(frames)

In [ ]:
# this is for testing if the merge was correct
#list(df_sp500[(df_sp500['Publish Date']== '2021-05-06') & (df_sp500['Ticker']== 'AES')].date)

In [ ]:
df_sp500.to_csv('data/sp500_fundamental_dailyprice.csv',index=False)

### Part 3: Append additional data source

#### 1.Industry/Sector

In [2]:
# load fundamental data
df_sp500=pd.read_csv('data/sp500_fundamental_dailyprice.csv')


In [3]:
df_industry_sector=pd.DataFrame(list(df_sp500.Ticker.unique()),columns=['Ticker'])

In [15]:
test=df_industry_sector.iloc[:5]

In [23]:
df2 = {'Ticker': 'KK'}
test = test.append(df2, ignore_index = True)
test

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
5,TTWO
6,KK


In [ ]:
sector_df= pd.DataFrame(columns=['Ticker', 'Sector', 'Industry'])

for ticker in df_industry_sector.Ticker.unique():
    try:
        sector_df=sector_df.append({
            'Ticker':ticker,
            'Sector':si.get_company_info(ticker).loc['sector'].Value,
            'Industry':si.get_company_info(ticker).loc['industry'].Value
        }, ignore_index=True)
        # print(ticker)
    except:
        print(f"No industry data available for {ticker}")


In [28]:
sector_df.tail()

,Ticker,Sector,Industry
408,XYL,Industrials,Specialty Industrial Machinery
409,YUM,Consumer Cyclical,Restaurants
410,ZBH,Healthcare,Medical Devices
411,ZBRA,Technology,Communication Equipment
412,ZTS,Healthcare,Drug Manufacturers—Specialty & Generic


In [30]:
sector_df.to_csv('data/industry_sector.csv',index=False)

In [31]:
df_sp500=pd.merge(df_sp500,sector_df,how='left', on='Ticker')

In [33]:
len(df_industry_sector.Ticker.unique())

413

#### 2.Earning Surprise

In [110]:
dftest=pd.DataFrame.from_dict(si.get_earnings_history("TSLA"))[['ticker','startdatetime','epsestimate','epsactual','epssurprisepct']]
dftest.startdatetime=pd.to_datetime(dftest.startdatetime).dt.date
dftest=dftest[dftest['startdatetime']>pd.to_datetime("2016-1-1").date()]
dftest['earningestimate_year']= pd.DatetimeIndex(dftest['startdatetime']).year
dftest['earningestimate_month']= pd.DatetimeIndex(dftest['startdatetime']).month
dftest.sort_values(by=['startdatetime'],inplace=True)

In [111]:
dftest['epsestimate_nextday'] = (pd.to_datetime(dftest['startdatetime']) + pd.DateOffset(days=1)).dt.date

In [112]:
dftest

,ticker,startdatetime,epsestimate,epsactual,epssurprisepct,earningestimate_year,earningestimate_month,epsestimate_nextday
26,TSLA,2016-02-10,0.10,-0.87,-1006.25,2016,2,2016-02-11
25,TSLA,2016-05-04,-0.58,-1.45,-148.29,2016,5,2016-05-05
24,TSLA,2016-08-03,-0.51,-1.61,-218.18,2016,8,2016-08-04
23,TSLA,2016-10-26,-0.54,0.71,232.22,2016,10,2016-10-27
22,TSLA,2017-02-22,-0.43,-0.69,-60.84,2017,2,2017-02-23
21,TSLA,2017-05-03,-0.81,-1.33,-63.79,2017,5,2017-05-04
20,TSLA,2017-08-02,-1.82,-1.33,27.04,2017,8,2017-08-03
19,TSLA,2017-11-01,-2.20,-2.92,-32.49,2017,11,2017-11-02
18,TSLA,2018-02-07,-3.09,-3.04,1.68,2018,2,2018-02-08
17,TSLA,2018-05-02,-3.58,-3.35,6.48,2018,5,2018-05-03


In [109]:
df_sp500[df_sp500['Ticker']=='TSLA']['Publish Date'].unique()

array(['2016-08-05', '2016-11-02', '2017-03-01', '2017-05-10',
       '2017-08-04', '2017-11-03', '2018-02-23', '2018-05-07',
       '2018-08-06', '2018-11-02', '2019-02-19', '2019-04-29',
       '2019-07-29', '2019-10-29', '2020-04-28', '2020-04-30',
       '2020-07-28', '2020-10-26', '2021-02-08', '2021-04-28'],
      dtype=object)

In [ ]:
### Part 4: Target Data 

In [ ]:
# tickers_sp500
frames=[]
for ticker in ['AAPL','TSLA']:
    try:
        df1=df_fundamentals[df_fundamentals['Ticker']==ticker]['']
        df2=si.get_data(ticker , start_date = start_date, end_date=end_date,index_as_date=False)
    except:
        print(f"No data available for {ticker}")
    else:
        df_merge=pd.merge(df1,df2,how='left',left_on='Ticker',right_on='ticker')
        df_merge= df_merge[(df_merge.date > df_merge['Publish Date']) & (df_merge.date <= df_merge['Next_publish_date'])]
        frames.append(df_merge)
    

df_sp500 = pd.concat(frames)

In [121]:
test=df_sp500[df_sp500['Ticker']=='TSLA'][['Ticker','Publish Date','date','close']]

In [122]:
test['close_minus1']=test['close'].shift(1)
test['close_plus1']=test['close'].shift(-4)
test

,Ticker,Publish Date,date,close,close_minus1
446158,TSLA,2016-08-05,2016-08-08,45.231998,NaN
446159,TSLA,2016-08-05,2016-08-09,45.816002,45.231998
446160,TSLA,2016-08-05,2016-08-10,45.130001,45.816002
446161,TSLA,2016-08-05,2016-08-11,44.981998,45.130001
446162,TSLA,2016-08-05,2016-08-12,45.122002,44.981998
...,...,...,...,...,...
447405,TSLA,2021-04-28,2021-07-22,649.260010,655.289978
447406,TSLA,2021-04-28,2021-07-23,643.380005,649.260010
447407,TSLA,2021-04-28,2021-07-26,657.619995,643.380005
447408,TSLA,2021-04-28,2021-07-27,644.780029,657.619995


In [ ]:
test['T0'] = (pd.to_datetime(test['Publish Date']) + pd.DateOffset(days=-1)).dt.date

In [125]:
df_sp500[df_sp500['Ticker']=='A']

,Ticker,Fiscal Year,Fiscal Period,Report Date,Publish Date,Revenue,Gross Profit,Operating Income (Loss),Net Income,Shares (Basic),...,date,open,high,low,close,adjclose,volume,ticker,Sector,Industry
0,A,2016,Q3,2016-07-31,2016-09-07,1.044000e+09,542000000.0,146000000.0,124000000,325000000.0,...,2016-09-08,47.070000,47.160000,46.939999,47.020000,44.856049,884700,A,Healthcare,Diagnostics & Research
1,A,2016,Q3,2016-07-31,2016-09-07,1.044000e+09,542000000.0,146000000.0,124000000,325000000.0,...,2016-09-09,46.509998,46.529999,44.869999,44.880001,42.814533,2507000,A,Healthcare,Diagnostics & Research
2,A,2016,Q3,2016-07-31,2016-09-07,1.044000e+09,542000000.0,146000000.0,124000000,325000000.0,...,2016-09-12,44.590000,45.810001,44.470001,45.750000,43.644493,1835400,A,Healthcare,Diagnostics & Research
3,A,2016,Q3,2016-07-31,2016-09-07,1.044000e+09,542000000.0,146000000.0,124000000,325000000.0,...,2016-09-13,45.360001,45.419998,44.610001,44.900002,42.833618,3461800,A,Healthcare,Diagnostics & Research
4,A,2016,Q3,2016-07-31,2016-09-07,1.044000e+09,542000000.0,146000000.0,124000000,325000000.0,...,2016-09-14,45.009998,45.099998,44.599998,44.860001,42.795460,1604800,A,Healthcare,Diagnostics & Research
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1250,A,2021,Q2,2021-04-30,2021-06-01,1.525000e+09,817000000.0,288000000.0,216000000,306000000.0,...,2021-08-26,173.389999,174.470001,172.100006,173.589996,172.874634,1273400,A,Healthcare,Diagnostics & Research
1251,A,2021,Q2,2021-04-30,2021-06-01,1.525000e+09,817000000.0,288000000.0,216000000,306000000.0,...,2021-08-27,174.029999,175.100006,172.990005,173.809998,173.093750,1185800,A,Healthcare,Diagnostics & Research
1252,A,2021,Q2,2021-04-30,2021-06-01,1.525000e+09,817000000.0,288000000.0,216000000,306000000.0,...,2021-08-30,173.860001,177.119995,173.860001,176.729996,176.001694,1221400,A,Healthcare,Diagnostics & Research
1253,A,2021,Q2,2021-04-30,2021-06-01,1.525000e+09,817000000.0,288000000.0,216000000,306000000.0,...,2021-08-31,177.100006,177.190002,174.830002,175.470001,174.746918,1733200,A,Healthcare,Diagnostics & Research
